# A pytorch ML adapter demo

#### Preamble
Demo of a relatively small pytorch model.
This notebook uses the [ml_adapter_torch](../../env/ml_adapter_torch) dependencies.
To start: 
```
bin/jupyter_notebook env/ml_adapter_torch ml_adapter/torch_autoencoder
```

In [1]:
!python --version

Python 3.11.9


In [2]:
# the sdk profile used to connect
PROFILE='_default_'
LOG_LEVEL='INFO'
MODEL_NAME='autoencoderV1'
MODEL_VERSION='1.0.0'

In [3]:
# setup INFO logging to see http requests made.
import logging
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=LOG_LEVEL,
    datefmt='%Y-%m-%d %H:%M:%S'
)

## Pytorch example

A simple auto-encoder pytorch model.

In [4]:
from autoencoder import AutoEncoder

In [5]:
# we saved our model class in a `autoencoder.py` file
from IPython.display import Code, Markdown
display(Code(filename='autoencoder.py'))
from autoencoder import AutoEncoder

import torch

class AutoEncoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        """
        Create a simple AutoEncoder
        """
        # Use an AutoEncoder and try to reconstruct both signals (the 20 samples back)
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(20, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 5),
        )

        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(5, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 20),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [6]:
# some pretrained weights
weights_path = 'AutoEncoderWeights.pth'

In [7]:
import torch
model = AutoEncoder()
model.load_state_dict(torch.load(weights_path))
model.eval()

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=5, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
  )
)

In [8]:
x_data = torch.randn(20, dtype=torch.float32)
x_data

tensor([-0.1321,  1.0263,  1.2909,  0.8727, -1.0449,  0.7338, -2.2298,  0.5470,
        -0.6775, -0.8363,  1.4936,  0.4840, -1.4861,  0.9581,  1.0316,  1.5117,
        -1.1868,  1.2446,  0.0825,  0.1862])

In [9]:
preds = model(x_data)
preds

tensor([ 0.0756,  0.1232,  0.0354,  0.0153,  0.1208,  0.0407,  0.0919,  0.0967,
         0.0121,  0.2355,  0.1027,  0.0877,  0.1641,  0.0342, -0.0093,  0.0025,
         0.0854, -0.1048,  0.0171,  0.1375], grad_fn=<ViewBackward0>)

## The adapter
The `V1TorchAdapter` from the `ml_adapter.torch` module wraps our model in a script that can be used as a waylay webscript or plug.

In [10]:
# choose a local archive location
ARCHIVE_LOC = 'autoencoder-pytorch'
# make sure its empty
!rm -fr autoencoder-pytorch

In [11]:
from ml_adapter.torch import V1TorchAdapter
# create an ML adapter to wrap our model
# by using a `weights.pt` postfix we are storing only the weights when serializing the model
MODEL_PATH='model-weights.pt'
adapter = V1TorchAdapter(model=model, model_path='model-weights.pt', location=ARCHIVE_LOC)

In [12]:
# the adapter exposes your model with a REST-compatible interface
result = await adapter.call({"instances": [ x_data.tolist() ]}) 
result

{'predictions': [[0.07556847482919693,
   0.12324532866477966,
   0.03544365242123604,
   0.015328444540500641,
   0.1207667887210846,
   0.04073508456349373,
   0.09185446798801422,
   0.09667737036943436,
   0.01210200134664774,
   0.2355400025844574,
   0.10270106792449951,
   0.08770085126161575,
   0.16406965255737305,
   0.03420832008123398,
   -0.00929244700819254,
   0.002525750547647476,
   0.08540024608373642,
   -0.10478955507278442,
   0.017097875475883484,
   0.13751526176929474]]}

In [16]:
# because we store only weights, the adapter archive needs to now about autoencode model class:
await adapter.add_script('autoencoder.py')

autoencoder.py <ml_adapter.base.assets.python.PythonScriptAsset>

### Creating the plug
Tell the adapter to configure itself as a plug: this generates a number of _assets_ that will be uploaded and define the plug behaviour
* a `plug.json` _manifest_ file that defines the name, version, _runtime_, deploy settings, metadata ...
* a `requirements.txt` package dependencies file
* a `main.py` webscript script
* additional scripts we added above, like the `autoencoder.py` and the `model-weights.pt`

In [17]:
## configure any 'memory' or 'cpu' deploy settings 
deploy_overrides = {'limits' : { 'memory': '2G' }, 'requests' : { 'memory' : '1G' }}
## configure the webscript to use our model
adapter = adapter.as_plug({
    'name': MODEL_NAME, 
    'description':'pytorch autoencoder for caats', 
    'deploy' : deploy_overrides                                                                                          
})

In [15]:
# del adapter.assets.children[1]

In [19]:
await adapter.save()
list(a.path for a in adapter.assets)

['openapi.json',
 'requirements.txt',
 'main.py',
 'model-weights.pt',
 'autoencoder.py',
 'plug.json']

In [20]:
# lets have a look at the generated python plug:
display(Code(filename=f'{ARCHIVE_LOC}/main.py'))

# You could adapt this script to have specific error handling or handling of request/response

# ml_adapter.torch.adapter.V1TorchAdapter model adapter
import os
from ml_adapter.api.data import v1 as V1
from ml_adapter.torch.adapter import V1TorchAdapter

# optional type alias for plug response
StatusAndRawData = tuple[str, V1.V1PredictionResponse|V1.V1ErrorResponse]

STATE_OK = 'PREDICTED'
STATE_NOK = 'FAILED'

MODEL_PATH = os.environ.get('MODEL_PATH', 'model-weights.pt')
MODEL_CLASS = os.environ.get('MODEL_CLASS', 'autoencoder.AutoEncoder')

# Initialize the model adapter.
# Provide a `model` argument if you want to create/load the model yourself.
adapter = V1TorchAdapter(
    model_path=MODEL_PATH, model_class=MODEL_CLASS
)

async def execute(properties: V1.V1Request, console, logger) -> StatusAndRawData:
    try:
        result = await adapter.call(properties)
        return (STATE_OK, result)
    except Exception as err:
        logger.exception(err)
        error_message = str(err)
        console.error(error_message)
        return (STATE_NOK, { 'error': error_message, 'predictions': [] })

In [22]:
# once stored, the adapter can be restored later with
# adapter = await V1TorchAdapter(model_path='model-weights.pt', model_class=AutoEncoder, location=ARCHIVE_LOC).load()

In [22]:
list(a.path for a in adapter.assets)

['openapi.json',
 'requirements.txt',
 'main.py',
 'model-weights.pt',
 'autoencoder.py',
 'plug.json']

### Uploading the plug using the SDK
To upload these assets and create a plug, we need to call the [create plug](https://docs.waylay.io/openapi/public/redocly/registry.html#tag/Plugs/operation/create_plugs) REST api. 

The code belows uses the `ml_tool` plugin to handle this.
Alternatively you could call `await adapter.save_archive()`
which creates an `autoencoder-pytorch.tar.gz` archive that you can upload as a plug with `curl` or using the waylay console.

In [23]:
from waylay.sdk import WaylayClient

In [24]:
# check the SDK client
client = WaylayClient.from_profile('staging')

In [25]:
ref = await client.ml_tool.create_plug(adapter)
ref

2024-06-12 14:00:15 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/accounts/v1/tokens?grant_type=client_credentials "HTTP/1.1 200 OK"
2024-06-12 14:00:15 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/registry/v2/plugs/?draft=false&comment=&async=true "HTTP/1.1 202 Accepted"


{'message': 'Building and deploying plug autoencoderV1@0.0.1',
 '_links': {'event': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$zrLKLX-4edcrChChVBWrB&children=true'},
  'job': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/verify/740799ef-d515-4704-8718-903851c9899e$zrLKLX-4edcrChChVBWrB'}},
 'entity': {'createdBy': 'users/edb8841f-122e-4f7d-a412-397764bc9996',
  'createdAt': '2024-06-12T12:00:15.988Z',
  'updatedBy': 'users/edb8841f-122e-4f7d-a412-397764bc9996',
  'updatedAt': '2024-06-12T12:00:16.017Z',
  'updates': [{'operation': 'create',
    'at': '2024-06-12T12:00:16.017Z',
    'by': 'users/edb8841f-122e-4f7d-a412-397764bc9996',
    'comment': '',
    'jobs': ['740799ef-d515-4704-8718-903851c9899e$zrLKLX-4edcrChChVBWrB',
     '740799ef-d515-4704-8718-903851c9899e$zqBjfaxXehToOTcDXA-t2',
     '740799ef-d515-4704-8718-903851c9899e$QD6cVj3WJh6_AQkrdvb6A']}],
  'status': 'pending',
  'runtime': {'deprec

In [26]:
# wait until the build, deploy and verify jobs for the webscript have finished
# NOTE: this might take quite a few minutes, as the dependencies for torch webscripts are quite big and the images not yet optimised
ref = await client.ml_tool.wait_until_ready(ref)

2024-06-12 14:02:51 INFO     Waiting for autoencoderV1@0.0.1 to be ready:
2024-06-12 14:02:51 INFO     listening on https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$zrLKLX-4edcrChChVBWrB&children=true
2024-06-12 14:02:51 INFO     HTTP Request: GET https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$zrLKLX-4edcrChChVBWrB&children=true "HTTP/1.1 200 OK"
2024-06-12 14:02:51 INFO     ack: Listening to events of jobs dependent on job 740799ef-d515-4704-8718-903851c9899e$zrLKLX-4edcrChChVBWrB
2024-06-12 14:02:51 INFO     autoencoderV1@0.0.1 build: active
2024-06-12 14:02:51 INFO     autoencoderV1@0.0.1 deploy: waiting-children
2024-06-12 14:02:51 INFO     autoencoderV1@0.0.1 verify: waiting-children
2024-06-12 14:03:04 INFO     keep-alive: {}
2024-06-12 14:03:34 INFO     keep-alive: {}
2024-06-12 14:04:04 INFO     keep-alive: {}
2024-06-12 14:04:34 INFO     keep-alive: {}
2024-06-12

In [27]:
# test the webscript invocation
await client.ml_tool.test_plug(ref, x_data.tolist())

2024-06-12 14:21:13 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/rules/v1/sensors/autoencoderV1/versions/0.0.1 "HTTP/1.1 200 OK"


[0.07556850463151932,
 0.12324536591768265,
 0.035443682223558426,
 0.015328466892242432,
 0.12076683342456818,
 0.04073513671755791,
 0.09185449779033661,
 0.09667740762233734,
 0.01210204977542162,
 0.23554003238677979,
 0.10270112752914429,
 0.08770088851451874,
 0.16406969726085663,
 0.03420836478471756,
 -0.009292409755289555,
 0.002525780349969864,
 0.08540026843547821,
 -0.10478952527046204,
 0.01709790527820587,
 0.13751532137393951]

In [28]:
# remove the webscript
await client.ml_tool.remove(ref)

2024-06-12 14:22:00 INFO     HTTP Request: DELETE https://api-aws-dev.waylay.io/registry/v2/plugs/autoencoderV1/versions/0.0.1?force=true "HTTP/1.1 202 Accepted"


{'message': 'Removing plug version autoencoderV1@0.0.1',
 '_links': {'event': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=undeploy&id=740799ef-d515-4704-8718-903851c9899e$T5YgeLbiSiHJFPSvUBAjf&children=true'},
  'job': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/undeploy/740799ef-d515-4704-8718-903851c9899e$T5YgeLbiSiHJFPSvUBAjf'}},
 'versions': ['0.0.1']}

#### About `ml_tool`
The `client.ml_tool` methods are essentialy wrappers around the methods of the [registry](https://docs.waylay.io/openapi/public/redocly/registry.html#tag/plugs) service. Alternatively you can use the [`client.registry.plug`](https://github.com/waylayio/waylay-sdk-registry-py) methods directly.